In [ ]:
from typing import Dict
from dotenv import load_dotenv
from langchain.agents import  AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

load_dotenv()

In [ ]:
prompt = """
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}

"""

prompt = PromptTemplate.from_template(prompt)


In [ ]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools.render import  render_text_description

# Define the tools
@tool
def characters_count(word: str) -> Dict[str, int]:
    """This tool count the number of characters in a word and returns the number. use this tool only if the intent of the user is to count characters"""
    return { "response": len(word) }

tools = [characters_count, DuckDuckGoSearchRun(description="This is a search engine tool, If you don't know a piece of information use this tool to search for it don't count on previous knowledge. use this tool to get information from the web.")]

print(render_text_description(tools).replace("\n", "\n\n"))


In [ ]:
prompt = prompt.partial(
    tools=render_text_description(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

In [ ]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)


In [ ]:
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
output_parser = ReActSingleInputOutputParser()
llm_with_stop = llm.bind(stop=["\nObservation"])
agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt
    | llm_with_stop
    | output_parser
)


In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "how many characters are in the middle name of will smith",
                       "chat_history":[]})